In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np; np.set_printoptions(linewidth=110); print(np.get_printoptions()['linewidth'])
# first we need a bit of import boilerplate
sys.path.append('/your/path/')
manifest_path = '/your/path/Data/Allen_Institute_Dataset/manifest.json'
project_dir = '/your/path/'

import os
import sys
import glob
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import trange
import time

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession,
    removed_unused_stimulus_presentation_columns
)
from allensdk.brain_observatory.ecephys.visualization import (
    plot_mean_waveforms, 
    plot_spike_counts, 
    raster_plot)
from allensdk.brain_observatory.visualization import plot_running_speed

pd.set_option("display.max_columns", None)

import data_visualizer
import data_model
import hierarchical_sampling_model
import util
import smoothing_spline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


110


In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

Download time: 720000


# All animals.

In [57]:
session_table = cache.get_session_table()
display(session_table)

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"
739448407,2019-10-03T00:00:00Z,716813543,brain_observatory_1.1,112.0,M,wt/wt,625,2221,6,"[grey, VISam, nan, VIS, VISp, VISl, VISrl]"
742951821,2019-10-03T00:00:00Z,723627604,brain_observatory_1.1,120.0,M,wt/wt,893,2219,6,"[VISal, nan, grey, VISl, VISrl, VISp, VISpm, VIS]"
743475441,2019-10-03T00:00:00Z,722882755,brain_observatory_1.1,121.0,M,wt/wt,553,2225,6,"[LP, LGd, HPF, DG, CA3, CA1, VISrl, nan, PP, P..."
744228101,2019-10-03T00:00:00Z,719817805,brain_observatory_1.1,122.0,M,wt/wt,659,2226,6,"[Eth, TH, LP, POL, APN, DG, CA1, VIS, nan, CA3..."


# Condition metrics -- brain_observatory_1.1

In [14]:
session_id = 798911424
session = cache.get_session_data(session_id)

In [15]:
model = hierarchical_sampling_model.HierarchicalSamplingModel(session)
model.print_session_info()

{'specimen_name': 'Vip-IRES-Cre;Ai32-421338', 'age_in_days': 110.0, 'full_genotype': 'Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt', 'strain': 'C57BL/6J', 'sex': 'F', 'stimulus_name': 'brain_observatory_1.1', 'subject_id': '775876808', 'age': 'P110D', 'species': 'Mus musculus'}
num units: 825


In [16]:
print(session_id)
print(session.stimulus_names)

798911424
['spontaneous', 'gabors', 'flashes', 'drifting_gratings', 'natural_movie_three', 'natural_movie_one', 'static_gratings', 'natural_scenes', 'drifting_gratings_contrast']


#### drifting_gratings

In [17]:
stimulus_table = session.get_stimulus_table(['drifting_gratings'])
display(stimulus_table)

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3798,0.8,0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1586.180717,2,drifting_gratings,1588.182387,15,2.00167,246
3799,0.8,180,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1589.183244,2,drifting_gratings,1591.184924,2,2.00168,247
3800,0.8,45,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1592.185737,2,drifting_gratings,1594.187427,1,2.00169,248
3801,0.8,90,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1595.188274,2,drifting_gratings,1597.189944,8,2.00167,249
3802,0.8,270,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1598.190777,2,drifting_gratings,1600.192447,2,2.00167,250
...,...,...,...,...,...,...,...,...,...,...,...,...
49428,0.8,315,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,5384.355314,7,drifting_gratings,5386.356974,4,2.00166,260
49429,0.8,0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,5387.357797,7,drifting_gratings,5389.359487,1,2.00169,252
49430,0.8,90,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,5390.360334,7,drifting_gratings,5392.361994,8,2.00166,249


#### drifting_gratings_contrast

In [18]:
stimulus_table = session.get_stimulus_table(['drifting_gratings_contrast'])
display(stimulus_table)

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
70391,0.08,90,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9182.529857,15,drifting_gratings_contrast,9183.030265,2,0.500408,5027
70392,0.08,0,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9183.530674,15,drifting_gratings_contrast,9184.031095,2,0.500422,5028
70393,0.04,45,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9184.531517,15,drifting_gratings_contrast,9185.031935,2,0.500418,5029
70394,0.01,45,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9185.532354,15,drifting_gratings_contrast,9186.032765,2,0.500412,5030
70395,0.04,90,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9186.533177,15,drifting_gratings_contrast,9187.033600,2,0.500423,5031
...,...,...,...,...,...,...,...,...,...,...,...,...
70926,0.35,45,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9717.994044,15,drifting_gratings_contrast,9718.494475,2,0.500432,5059
70927,1,90,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9718.994907,15,drifting_gratings_contrast,9719.495310,2,0.500403,5061
70928,0.08,0,"[19258.96666667, 19258.96666667]","[250.0, 250.0]",0.04,9719.995714,15,drifting_gratings_contrast,9720.496125,2,0.500412,5028


#### flashes

In [19]:
stimulus_table = session.get_stimulus_table(['flashes'])
display(stimulus_table)

,color,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3647,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1285.929787,1,flashes,1286.179996,0.250209,244
3648,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1287.931477,1,flashes,1288.181681,0.250204,244
3649,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1289.933157,1,flashes,1290.183361,0.250204,244
3650,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1291.934847,1,flashes,1292.185046,0.250199,244
3651,1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1293.936507,1,flashes,1294.186708,0.250202,245
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1576.172387,1,flashes,1576.422593,0.250207,245
3793,1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1578.174047,1,flashes,1578.424256,0.250209,245
3794,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1580.175687,1,flashes,1580.425903,0.250217,244


#### gabors

In [20]:
stimulus_table = session.get_stimulus_table(['gabors'])
display(stimulus_table)

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,x_position,y_position,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,,,
1,0.8,90,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,84.942787,0,gabors,85.176306,4,20,-10,0.233519,1
2,0.8,0,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,85.176306,0,gabors,85.426505,4,30,40,0.250199,2
3,0.8,45,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,85.426505,0,gabors,85.676704,4,40,-30,0.250199,3
4,0.8,90,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,85.676704,0,gabors,85.926904,4,0,-40,0.250199,4
5,0.8,45,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,85.926904,0,gabors,86.177122,4,-40,10,0.250218,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,0.8,90,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,995.687177,0,gabors,995.937391,4,-30,-30,0.250214,124
3642,0.8,90,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,995.937391,0,gabors,996.187605,4,-10,-10,0.250214,32
3643,0.8,0,"[3644.93333333, 3644.93333333]","[20.0, 20.0]",0.08,996.187605,0,gabors,996.437819,4,40,-10,0.250214,60


#### static_gratings

In [21]:
stimulus_table = session.get_stimulus_table(['static_gratings'])
display(stimulus_table)

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,
49434,0.8,150,0.5,"[250.0, 250.0]",0.02,5399.367847,8,static_gratings,5399.618056,0.250209,4787
49435,0.8,0,0.5,"[250.0, 250.0]",0.16,5399.618056,8,static_gratings,5399.868265,0.250209,4788
49436,0.8,30,0.25,"[250.0, 250.0]",0.04,5399.868265,8,static_gratings,5400.118474,0.250209,4789
49437,0.8,60,0.75,"[250.0, 250.0]",0.04,5400.118474,8,static_gratings,5400.368684,0.250209,4790
49438,0.8,60,0.0,"[250.0, 250.0]",0.08,5400.368684,8,static_gratings,5400.618889,0.250206,4791
...,...,...,...,...,...,...,...,...,...,...,...
70385,0.8,150,0.5,"[250.0, 250.0]",0.08,9151.253694,14,static_gratings,9151.503914,0.250219,4895
70386,0.8,60,0.0,"[250.0, 250.0]",0.08,9151.503914,14,static_gratings,9151.754127,0.250213,4791
70387,0.8,0,0.75,"[250.0, 250.0]",0.04,9151.754127,14,static_gratings,9152.004340,0.250213,4800


# Condition metrics -- functional_connectivity

In [ ]:
session_id = 771160300
session = cache.get_session_data(session_id)

In [7]:
print(session_id)
print(session.stimulus_names)

['spontaneous',
 'gabors',
 'flashes',
 'drifting_gratings_contrast',
 'natural_movie_one_more_repeats',
 'natural_movie_one_shuffled',
 'drifting_gratings_75_repeats',
 'dot_motion']

#### drifting_gratings_75_repeats

In [13]:
stimulus_table = session.get_stimulus_table(['drifting_gratings_75_repeats'])
display(stimulus_table)

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
40339,0.1,0,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,3498.389895,5,drifting_gratings_75_repeats,3500.391575,2,2.00168,2082
40340,0.8,135,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,3501.392396,5,drifting_gratings_75_repeats,3503.394076,2,2.00168,2083
40341,0.1,135,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,3504.394885,5,drifting_gratings_75_repeats,3506.396605,2,2.00172,2084
40342,0.1,45,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,3507.397426,5,drifting_gratings_75_repeats,3509.399096,2,2.00167,2085
40343,0.1,135,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,3510.399905,5,drifting_gratings_75_repeats,3512.401585,2,2.00168,2084
...,...,...,...,...,...,...,...,...,...,...,...,...
40935,0.8,0,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,7086.388106,6,drifting_gratings_75_repeats,7088.389776,2,2.00167,2086
40936,0.1,45,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,7089.390635,6,drifting_gratings_75_repeats,7091.392265,2,2.00163,2085
40937,0.1,90,"[14037.96666667, 14037.96666667]","[250.0, 250.0]",0.04,7092.393126,6,drifting_gratings_75_repeats,7094.394806,2,2.00168,2089


#### drifting_gratings_contrast

In [10]:
orientation = [0.,]
contrast = [1.,]
stimulus_table = session.get_stimulus_table(['drifting_gratings_contrast'])
trials_indices = stimulus_table[
    stimulus_table['orientation'].isin(orientation) & 
    stimulus_table['contrast'].isin(contrast)].index.values
print('num trials:', len(trials_indices))
display(stimulus_table)

num trials: 15


,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3798,0.2,0,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,1636.833745,2,drifting_gratings_contrast,1637.334160,2,0.500416,246
3799,0.04,45,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,1637.834576,2,drifting_gratings_contrast,1638.335000,2,0.500424,247
3800,0.35,90,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,1638.835425,2,drifting_gratings_contrast,1639.335835,2,0.500411,248
3801,0.01,0,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,1639.836246,2,drifting_gratings_contrast,1640.336665,2,0.500419,249
3802,0.01,135,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,1640.837085,2,drifting_gratings_contrast,1641.337505,2,0.500421,250
...,...,...,...,...,...,...,...,...,...,...,...,...
4333,0.6,90,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,2172.281466,2,drifting_gratings_contrast,2172.781905,2,0.500439,273
4334,0.35,135,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,2173.282345,2,drifting_gratings_contrast,2173.782745,2,0.500401,271
4335,0.08,45,"[4198.96666667, 4198.96666667]","[250.0, 250.0]",0.04,2174.283146,2,drifting_gratings_contrast,2174.783565,2,0.500419,264


#### flashes

In [11]:
stimulus_table = session.get_stimulus_table(['flashes'])
color = [-1.,]
trials_indices = stimulus_table[stimulus_table['color'].isin(color)].index.values
print('num trials:', len(trials_indices))
display(stimulus_table)

num trials: 75


,color,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3647,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1276.532445,1,flashes,1276.782660,0.250215,244
3648,1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1278.534145,1,flashes,1278.784348,0.250203,245
3649,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1280.535815,1,flashes,1280.786025,0.250210,244
3650,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1282.537465,1,flashes,1282.787680,0.250215,244
3651,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1284.539125,1,flashes,1284.789343,0.250218,244
...,...,...,...,...,...,...,...,...,...,...,...,...
3792,1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1566.775145,1,flashes,1567.025355,0.250210,245
3793,1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1568.776815,1,flashes,1569.027025,0.250210,245
3794,-1,0.8,0,"[0.0, 0.0]","[300.0, 300.0]","[0.0, 0.0]",1570.778485,1,flashes,1571.028698,0.250213,244


#### dot motion.

In [12]:
stimulus_table = session.get_stimulus_table(['dot_motion'])
display(stimulus_table)

,Dir,Speed,coherence,contrast,start_time,stimulus_block,stimulus_name,stop_time,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,
76941,90,0.0005,0.9,1,8302.404245,9,dot_motion,8303.405096,1.000852,2090
76942,45,0,0.9,1,8304.405925,9,dot_motion,8305.406776,1.000852,2091
76943,45,0.001,0.9,1,8306.407605,9,dot_motion,8307.408436,1.000832,2092
76944,-45,0.01,0.9,1,8308.409275,9,dot_motion,8309.410116,1.000842,2093
76945,90,0.02,0.9,1,8310.410935,9,dot_motion,8311.411786,1.000852,2094
...,...,...,...,...,...,...,...,...,...,...
77356,45,0.0005,0.9,1,9133.098715,9,dot_motion,9134.099536,1.000822,2101
77357,0,0.001,0.9,1,9135.100385,9,dot_motion,9136.101226,1.000842,2113
77358,45,0.01,0.9,1,9137.102055,9,dot_motion,9138.102876,1.000822,2105


# Individual animal

In [4]:
session_id = 798911424
session = cache.get_session_data(session_id)

In [5]:
model = hierarchical_sampling_model.HierarchicalSamplingModel(session)
data_model = data_model.AllenInstituteDataModel(session)

In [6]:
model.print_session_info()

{'specimen_name': 'Vip-IRES-Cre;Ai32-421338', 'age_in_days': 110.0, 'full_genotype': 'Vip-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt', 'strain': 'C57BL/6J', 'sex': 'F', 'stimulus_name': 'brain_observatory_1.1', 'subject_id': '775876808', 'age': 'P110D', 'species': 'Mus musculus'}
num units: 825


### Neurons Selection

In [12]:
all_areas_map = session.map_probe_to_ecephys_structure_acronym(visual_only=False, verbose=True)
vis_areas_map = session.map_probe_to_ecephys_structure_acronym(visual_only=True, verbose=True)

----------- All recorded regions by probe ----------
probeA: ['APN' 'NOT' 'MB' 'DG' 'CA1' 'VISam']
probeB: ['LP' 'DG' 'CA1' 'VISam']
probeC: ['DG' 'ProS' 'SUB' 'VISp']
probeD: ['CA3' 'CA1' 'VISl']
probeE: ['LGv' 'CA3' 'CA2' 'CA1' 'VISal']
probeF: ['APN' 'TH' 'Eth' 'LP' 'CA3' 'DG' 'CA1' 'VISrl']
----------- Only visual ----------
probeA: ['VISam']
probeB: ['VISam']
probeC: ['VISp']
probeD: ['VISl']
probeE: ['VISal']
probeF: ['VISrl']


In [13]:
probes = ['probeC', 'probeD', 'probeE']
selected_units = session.units[
    session.units['ecephys_structure_acronym'].isin(util.VISUAL_AREA) &
    session.units['probe_description'].isin(probes)]
display(selected_units['probe_description'].value_counts())
display(selected_units['probe_description'].count())

probeC    94
probeE    89
probeD    78
Name: probe_description, dtype: int64

261

### Trials Selection manually (optional or skip)

In [226]:
# drifting gratings
orientation = [180, 225., 270.]  # 'null'
temporal_frequency = [4., 8.]  # 'null'

stimulus_table = session.get_stimulus_table(['drifting_gratings'])
trials_df = stimulus_table[
    stimulus_table['orientation'].isin(orientation) & 
    stimulus_table['temporal_frequency'].isin(temporal_frequency)]

all_trials_indices = trials_df.index.values
trials_groups = trials_df.groupby('stimulus_condition_id')
model.print_conditions(trials_groups)

total num trials: 90
0 251 15 225.0 4.0 [ 3804  3818  3861  3898  3976 31011 31016 31106 31116 49216 49298 49323 49328 49345 49363]
1 255 15 270.0 8.0 [ 3808  3911  3990  3991  3993 31027 31033 31049 31075 31077 31104 31167 31184 49272 49386]
2 261 15 180.0 8.0 [ 3815  3817  3829  3859 31062 31147 31162 31179 49222 49266 49296 49317 49329 49397 49431]
3 270 15 270.0 4.0 [ 3830  3841  3848  3901  3988 31042 49214 49220 49237 49294 49331 49334 49401 49429 49430]
4 278 15 225.0 8.0 [ 3857  3863  3931 31018 31087 31153 31188 31191 31193 49208 49351 49360 49373 49375 49418]
5 284 15 180.0 4.0 [ 3880  3905  3942  3955  3961 31007 31039 31102 31157 31172 31175 49206 49229 49238 49364]


In [8]:
condition_ids = [275, 268, 270, 284, 274, 249, 261, 278, 280, 256, 260, 257, 281]
stimulus_table = session.get_stimulus_table(['drifting_gratings'])
trials_df = stimulus_table[stimulus_table['stimulus_condition_id'].isin(condition_ids)]
all_trials_indices = trials_df.index.values
trials_groups = trials_df.groupby('stimulus_condition_id')
model.print_conditions(trials_groups)

0  249 temp freq [8.0] orient [90.0] contrast [0.8] [ 3801  3810  3828  3911  3964 31002 31054 31055 31178 49257 49317 49341 49365 49395 49430]
1  256 temp freq [15.0] orient [270.0] contrast [0.8] [ 3811  3845  3942  3958 31003 31056 31112 31126 31137 31169 31191 49260 49305 49344 49381]
2  257 temp freq [8.0] orient [315.0] contrast [0.8] [ 3813  3822  3870  3876  3932  3936  3987 31022 31027 31075 31171 49215 49217 49368 49370]
3  260 temp freq [4.0] orient [315.0] contrast [0.8] [ 3816  3888  3930  3943  3948 31028 31033 31190 49263 49266 49320 49364 49410 49411 49428]
4  261 temp freq [8.0] orient [135.0] contrast [0.8] [ 3817  3826  3875  3928 31020 31053 31129 31161 31162 49230 49254 49268 49275 49313 49407]
5  268 temp freq [4.0] orient [45.0] contrast [0.8] [ 3833  3882  3884  3899  3951  3988 31040 31086 31089 31116 31149 31197 49255 49277 49327]
6  270 temp freq [8.0] orient [45.0] contrast [0.8] [ 3835  3898  3914  3934 31062 31104 31114 49250 49274 49281 49286 49304 49322 

### Select all conditions

In [10]:
stimulus_type = 'drifting_gratings'
stimulus_table = session.get_stimulus_table([stimulus_type])
display(stimulus_table)

trials_groups = stimulus_table.groupby('stimulus_condition_id')
all_condition_ids = stimulus_table['stimulus_condition_id'].unique()
all_trials_indices = stimulus_table.index.values
print('total num trials:', len(all_trials_indices))
model.print_conditions(trials_groups)
print('all condition ids:', all_condition_ids)

,contrast,orientation,phase,size,spatial_frequency,start_time,stimulus_block,stimulus_name,stop_time,temporal_frequency,duration,stimulus_condition_id
stimulus_presentation_id,,,,,,,,,,,,
3798,0.8,0.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1586.180717,2.0,drifting_gratings,1588.182387,15.0,2.00167,246
3799,0.8,180.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1589.183244,2.0,drifting_gratings,1591.184924,2.0,2.00168,247
3800,0.8,45.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1592.185737,2.0,drifting_gratings,1594.187427,1.0,2.00169,248
3801,0.8,90.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1595.188274,2.0,drifting_gratings,1597.189944,8.0,2.00167,249
3802,0.8,270.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,1598.190777,2.0,drifting_gratings,1600.192447,2.0,2.00167,250
...,...,...,...,...,...,...,...,...,...,...,...,...
49428,0.8,315.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,5384.355314,7.0,drifting_gratings,5386.356974,4.0,2.00166,260
49429,0.8,0.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,5387.357797,7.0,drifting_gratings,5389.359487,1.0,2.00169,252
49430,0.8,90.0,"[5308.98333333, 5308.98333333]","[250.0, 250.0]",0.04,5390.360334,7.0,drifting_gratings,5392.361994,8.0,2.00166,249


total num trials: 630
0  246 temp freq [15.0] orient [0.0] contrast [0.8] [ 3798  3852  3947  3966  3979 31016 31026 31160 31186 49206 49245 49346 49372 49409 49412]
1  247 temp freq [2.0] orient [180.0] contrast [0.8] [ 3799  3827  3847  3874  3883  3887  3908  3939 31059 31111 31115 49278 49348 49387 49396]
2  248 temp freq [1.0] orient [45.0] contrast [0.8] [ 3800  3807  3820  3853  3907  3975  3993 31013 31036 31091 31106 31120 31127 49216 49397]
3  249 temp freq [8.0] orient [90.0] contrast [0.8] [ 3801  3810  3828  3911  3964 31002 31054 31055 31178 49257 49317 49341 49365 49395 49430]
4  250 temp freq [2.0] orient [270.0] contrast [0.8] [ 3802  3903  3991 31006 31109 31136 49229 49267 49273 49300 49308 49363 49369 49379 49382]
5  251 temp freq [1.0] orient [90.0] contrast [0.8] [ 3803  3806  3885  3890  3983 31017 31052 31131 31132 49224 49334 49345 49351 49373 49384]
6  252 temp freq [1.0] orient [0.0] contrast [0.8] [ 3804  3869  3873  3916 31037 31050 31061 31157 31193 49271 